# Group-level satistical analysis of brain electrical properties  - Diagnostic tests
This notebooks provides the diagnostic tests performed to assess the robustness of the models used in the group-level satistical analysis of brain electrical properties ($\sigma$ and $\epsilon_r$).

Part of nano-eptk package by Arnaud Boutillon (arnaud.boutillon@kcl.ac.uk).

## Python environment setting

In [1]:
import os
import sys
import warnings

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.stats.diagnostic import het_breuschpagan, het_white

base_dir = os.path.abspath(os.path.join(os.path.pardir, os.path.pardir))

warnings.filterwarnings("ignore")

## 1. Neonatal data
Diagnostic tests were performed on subjects whose postmenstrual age (PMA) $\geq$ 37 weeks so that both term-born and preterm infants are considered at similar age at scan; $n = 534$ subjects were considered in total. 

In [2]:
# Load dataframe
csv_dir = os.path.join(base_dir, "data", "cohorts_csv")
df_path = os.path.join(csv_dir, "dHCP_filtered_MD_RVV_PMA.csv")
df = pd.read_csv(df_path)

# Display cohort size
print(f"dHCP sessions n = {len(df.index)}")
print(f"dHCP subjects m = {df['subject_id'].nunique()}")

dHCP sessions n = 534
dHCP subjects m = 534


## 2. Variance inflation factors (VIFs)
We computed the variance inflation factors (VIF) to measure the magnitude of multicollinearity between regressors: postmenstrual age (PMA), preterm birth (PTB), relative ventricle volume (RVV) and mean diffusivity (MD). All calculated VIF values were below the commonly accepted threshold of $5$, suggesting limited multicollinearity and reliable estimation of regression coefficients.

In [3]:
# Load dataframe
df = pd.read_csv(df_path)

# Data header
x_cols = ["PMA", "RVV", "brain_MD", "PTB"]

# For VIF
df = df[x_cols]
df = add_constant(df)

# Iter over regressors
for i in range(1, df.shape[1]):

    # Compute and display VIF
    vif = variance_inflation_factor(df.values, i)
    print(f"VIF {df.columns[i]}: {vif:.2f}")

VIF PMA: 1.37
VIF RVV: 1.24
VIF brain_MD: 1.66
VIF PTB: 1.51


## 3. Non-linear and interaction terms
Interaction terms between predictors were included in the multivariate model to assess, for example, whether prematurity had an non-linear effect on the relationship between conductivity and development (i.e. $\textrm{PMA} \times \textrm{PTB}$ interaction). Similarly, we tried incorporating quadratic and cubic terms (e.g. $\textrm{PMA}^2$ or $\textrm{PMA}^3$) in our models. However, none of these non-linear interaction resulted in a better model fit, so these interaction terms were discarded from the statistical analysis.

In [4]:
# Set file path
summary_dir = os.path.join(base_dir, "statsmodels")
summary_path = os.path.join(summary_dir, "diagnostic_experiment1.txt")

# Load dataframe
df = pd.read_csv(df_path) 

# Data header
y_col = "brain_SIG"
x_cols = ["PMA", "RVV", "brain_MD", "PTB"]

# Center data
for x_col in x_cols:
    df[x_col] -= df[x_col].mean()

# Fit linear model
formula = f"{y_col} ~  {' + '.join(x_cols)}"
lm = smf.ols(formula=formula, data=df).fit()

# Fit linear model with interaction
formula_inter = f"{y_col} ~  {' + '.join(x_cols)} + PMA*PTB"
lm_inter = smf.ols(formula=formula_inter, data=df).fit()

# Fit linear model with non-linear term
formula_nl = f"{y_col} ~  {' + '.join(x_cols)} + I(PMA**2)"
lm_nl = smf.ols(formula=formula_nl, data=df).fit()

# Initialize output directory
output_dir = os.path.dirname(summary_path)
os.makedirs(output_dir, exist_ok=True)

# Write summary
with open(summary_path, "w") as fh:
    fh.write(lm.summary().as_text())
    fh.write("\n\n")
    fh.write(lm_inter.summary().as_text())
    fh.write("\n\n")
    fh.write(lm_nl.summary().as_text())

# Model summary
f = open(summary_path, 'r')
file_contents = f.read()
print(file_contents)
f.close()

                            OLS Regression Results                            
Dep. Variable:              brain_SIG   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.348
Method:                 Least Squares   F-statistic:                     71.99
Date:                Thu, 07 Aug 2025   Prob (F-statistic):           1.12e-48
Time:                        19:31:20   Log-Likelihood:                 944.08
No. Observations:                 534   AIC:                            -1878.
Df Residuals:                     529   BIC:                            -1857.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1529      0.002    642.024      0.0

Both the $\textrm{PMA} \times \textrm{PTB}$ interaction and non-linear $\textrm{PMA}^2$ terms are non-significant (*p* = 0.230 and *p* = 0.458).

## 4. Heteroscedasticity test
Heteroscedasticity tests revealed no evidence of variance heterogeneity in the residuals. Consequently, we did not apply heteroscedasticity correction, as it is rarely used in conjunction with linear mixed-effects (LME) models.

In [5]:
# Load dataframe
df = pd.read_csv(df_path) 

# Data header
y_col = "brain_SIG"
x_cols = ["PMA", "RVV", "brain_MD", "PTB"]

# Center data
for x_col in x_cols:
    df[x_col] -= df[x_col].mean()

# Fit linear model
formula = f"{y_col} ~  {' + '.join(x_cols)}"
lm = smf.ols(formula=formula, data=df).fit()

# Residuals and design matrix (exog)
resid = lm.resid
exog = lm.model.exog

# Breusch-Pagan test
bp_tests = het_breuschpagan(resid, exog)
bp_labels = ['LM stat', 'LM p-value', 'F-stat', 'F p-value']
print("Breusch-Pagan test:")
for bp_label, bp_test in zip(bp_labels, bp_tests):
    print(f"{bp_label}: {bp_test:.4f}")

# White test 
hw_tests = het_white(resid, exog)
hw_labels = ['LM stat', 'LM p-value', 'F-stat', 'F p-value']
print("\nWhite test:")
for hw_label, hw_test in zip(hw_labels, hw_tests):
    print(f"{hw_label}: {hw_test:.4f}")

Breusch-Pagan test:
LM stat: 2.6770
LM p-value: 0.6133
F-stat: 0.6663
F p-value: 0.6156

White test:
LM stat: 20.8324
LM p-value: 0.1060
F-stat: 1.5049
F p-value: 0.1045


## 5. Impact of outliers

### 5.a. Robust linear models
We investigated the impact of outlier data points by employing separate robust linear models, which produced results consistent with those from LMEs. This suggested that any potential outliers were not influential enough to alter the model parameters. In other words, rather than incorporating robust estimators directly into mixed-effects models, we performed a diagnostic check by comparing our LME results to those from robust models.

In [6]:
# Set file path
summary_path = os.path.join(summary_dir, "diagnostic_experiment2.txt")

# Load dataframe
df = pd.read_csv(df_path) 

# Data header
y_col = "brain_SIG"
x_cols = ["PMA", "RVV", "brain_MD", "PTB"]

# Center data
for x_col in x_cols:
    df[x_col] -= df[x_col].mean()

# Fit linear model
formula = f"{y_col} ~  {' + '.join(x_cols)}"
lm = smf.ols(formula=formula, data=df).fit()

# Endog and exog variables
exog = df[x_cols]
exog = add_constant(exog)
endog = df[y_col]

# Fit robust model
rlm = sm.RLM(endog, exog, M=sm.robust.norms.HuberT()).fit()

# Initialize output directory
output_dir = os.path.dirname(summary_path)
os.makedirs(output_dir, exist_ok=True)

# Write summary
with open(summary_path, "w") as fh:
    fh.write(lm.summary().as_text())
    fh.write("\n\n")
    fh.write(rlm.summary().as_text())

# Model summary
f = open(summary_path, 'r')
file_contents = f.read()
print(file_contents)
f.close()

                            OLS Regression Results                            
Dep. Variable:              brain_SIG   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.348
Method:                 Least Squares   F-statistic:                     71.99
Date:                Thu, 07 Aug 2025   Prob (F-statistic):           1.12e-48
Time:                        19:31:20   Log-Likelihood:                 944.08
No. Observations:                 534   AIC:                            -1878.
Df Residuals:                     529   BIC:                            -1857.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1529      0.002    642.024      0.0

### 5.b. Leverage points
To assess the effect of potential leverage points ($x$-extrema points), we excluded points outside the $x$-range $1\%-99\%$ percentiles. The results were consistent with those of the LME models fitted to all data points, suggesting that no leverage point had corrupted our regression analysis.

In [7]:
# Data path
summary_path = os.path.join(summary_dir, "diagnostic_experiment3.txt")

# Load dataframe
df = pd.read_csv(df_path) 

# Date header
y_col = "brain_SIG"
x_cols = ["PMA", "RVV", "brain_MD", "PTB"]

# Center data
for x_col in x_cols:
    df[x_col] -= df[x_col].mean()

# Fit linear model
formula = f"{y_col} ~  {' + '.join(x_cols)}"
lm = smf.ols(formula=formula, data=df).fit()

# 1%-99% percentile for all regressors
q_lows = []
q_his = []
for x_col in x_cols:
    q_lows += [df[x_col].quantile(0.01)]
    q_his += [df[x_col].quantile(0.99)]

# Merge mask
mask = np.ones(len(df), dtype=bool)
for i, x_col in enumerate(x_cols):
    mask &= (df[x_col] > q_lows[i]) & (df[x_col] < q_his[i])
df = df[mask]

# Fit linear model
lm_lp = smf.ols(formula=formula, data=df).fit()

# Initialize output directory
output_dir = os.path.dirname(summary_path)
os.makedirs(output_dir, exist_ok=True)

# Write summary
with open(summary_path, "w") as fh:
    fh.write(lm.summary().as_text())
    fh.write("\n\n")
    fh.write(lm_lp.summary().as_text())

# Model summary
f = open(summary_path, 'r')
file_contents = f.read()
print(file_contents)
f.close()

                            OLS Regression Results                            
Dep. Variable:              brain_SIG   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.348
Method:                 Least Squares   F-statistic:                     71.99
Date:                Thu, 07 Aug 2025   Prob (F-statistic):           1.12e-48
Time:                        19:31:20   Log-Likelihood:                 944.08
No. Observations:                 534   AIC:                            -1878.
Df Residuals:                     529   BIC:                            -1857.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1529      0.002    642.024      0.0